# Trabalho Prático 0

## Introdução

A resolução deste trabalho prático tem como objetivo servir de iniciação à componente prática da unidade curricular de Estruturas criptográficas, onde se pretende:
   - Instalar as ferramentas computacionais necessárias para a realização dos trabalhos práticos.
   - Demonstrar pequenas aplicações implementadas em **Python** e em **SageMath**.

A aplicação em **Python** deve ser implementada de tal forma que permita a comunicação entre um emissor e um recetor, com as seguintes caracteristicas:
   - Criptograma e metadados devem ser autenticados.
   - Utilizar uma cifra simétrica em modo _stream cipher_.
   - Autenticar previamente a chave.
    
A aplicação em **SageMath** deve ser implementada de tal forma que:
   - Crie 4 corpos finitos primos.
   - Crie um _plot_ de uma função em cada um dos corpos finitos primos.
   - Teste, por amostragem, o facto de que, considerando um expoente _n_, um elemento primitivo de um corpo _g_ e um número primo _p_, se $g ^ n = 1$ , então $n = 0 mod (p-1)$.

## Aplicação Python

### Imports

Esta secção executa os _imports_ de **Python** que contêm as funções necessárias para desenvolver a aplicação enunciada.

In [1]:
import os
from getpass import getpass
from multiprocessing import Process,Pipe
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes, hmac
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.exceptions import InvalidSignature
from base64 import b64encode, b64decode

### Definição da classe de multiprocessamento

Esta secção tem o propósito de definir a classe de multiprocessamento, que permite uma comunicação bidireccional com o _Emitter_ e o _Receiver_, sendo estes dois processos criados e implementados pela **API** [**multiprocessing**](https://docs.python.org/2/library/multiprocessing.html).

In [2]:
class BiConnection(object):
    def __init__(self,left,right):
        left_side,right_side = Pipe()
        self.timeout = None
        self.left_process = Process(target=left,args=(left_side,))
        self.right_process = Process(target=right,args=(right_side,))
        self.left = lambda : left(left_side)
        self.right = lambda : right(right_side)
    
    def manual(self): # Execução manual apenas devido ao facto de a password ter que ser lida em ambos os lados do Pipe
        self.left()
        self.right()
    

### Definição do Emissor e do Recetor

Nesta secção encontram-se implementados os comportamentos do emissor e do recetor que participam na comunicação bidireccional da aplicação.

In [3]:
def Emissor(connection):
    con_salt = os.urandom(16)
    passwd = bytes(getpass('Password do emissor: '),'utf-8')
    text_to_send = os.urandom(128)
    print('Texto a cifrar e enviar:')
    print(b64encode(text_to_send))
    try:
        #derivar a chave apartir da password
        derivation = PBKDF2HMAC(
                algorithm = hashes.SHA256(),
                length = 96,
                salt = con_salt,
                iterations = 100000,
                backend = default_backend()
        )
        #Separar a password para cifragem e autenticação
        full_key = derivation.derive(passwd)
        cript_key = full_key[:32]
        mac_key = full_key[32:64]
        mac_passwd_key = full_key[64:]
        
        # Utilizar o AES com um dos modos que o torna numa stream cipher para cifrar o criptograma.
        nonce = os.urandom(16)
        cipher = Cipher(algorithm = algorithms.AES(cript_key),mode= modes.CTR(nonce),backend = default_backend())
        cryptogram = cipher.encryptor().update(text_to_send)
        
        #geração do código de autenticação do criptograma e dos metadados
        message_to_authenticate = cryptogram + nonce + con_salt
        hasher = hmac.HMAC(mac_key,hashes.SHA256(),default_backend())
        hasher.update(message_to_authenticate)
        hash_msg = hasher.finalize()
        
        #geração do código de autenticação da chave
        #esforço computacional é reduzido no recetor caso insira a password errada.
        hasher_passwd = hmac.HMAC(mac_passwd_key,hashes.SHA256(),default_backend())
        hasher_passwd.update(cript_key)
        hash_pass = hasher_passwd.finalize()
        obj = {'cryptogram': cryptogram, 'mac_code': hash_msg,'pass_code': hash_pass, 'salt': con_salt, 'nonce': nonce}
        connection.send(obj)
    except Exception as e:
        print(e)
def Recetor(connection):
    passwd = bytes(getpass('Password do recetor: '), 'utf-8')
    try:
        obj = connection.recv()
        
        #Obter parâmetros no objeto
        pass_code = obj['pass_code']
        cryptogram = obj['cryptogram']
        mac_code = obj['mac_code']
        salt = obj['salt']
        nonce = obj['nonce']
        
        #derivar a chave apartir da password lida
        derivation = PBKDF2HMAC(
                algorithm = hashes.SHA256(),
                length = 96,
                salt = salt,
                iterations = 100000,
                backend = default_backend()
        )
        #Separar a password para cifragem e autenticação
        full_key = derivation.derive(passwd)
        cript_key = full_key[:32]
        mac_key = full_key[32:64]
        mac_passwd_key = full_key[64:]
        
        
        
        #Autenticação prévia da chave
        hasher_passwd = hmac.HMAC(mac_passwd_key,hashes.SHA256(),default_backend())
        hasher_passwd.update(cript_key)
        hasher_passwd.verify(pass_code)
        
        #Autenticação do criptograma e metadados
        message_to_authenticate = cryptogram + nonce + salt
        hash_msg = hmac.HMAC(mac_key,hashes.SHA256(),default_backend())
        hash_msg.update(message_to_authenticate)
        try:
            hash_msg.verify(mac_code)
            
            #Decifrar criptograma
            cipher = Cipher(algorithm = algorithms.AES(cript_key),mode = modes.CTR(nonce),backend = default_backend())
            plain_text = cipher.decryptor().update(cryptogram)
            print('Texto decifrado:')
            print(b64encode(plain_text))
        except InvalidSignature as i:
            print("Código de autenticação não é válido!")
    except Exception as e:
        print(e)
    connection.close()
        
        


### Iniciação do processo

Por último, resta apenas criar um objeto de conexão bidireccional passando-lhe como argumentos o emissor e o recetor definidos e, finalmente, prosseguindo com a execução de ambos os processos através da chamada à função manual.

In [4]:
BiConnection(Emissor,Recetor).manual()

Password do emissor: ········
Texto a cifrar e enviar:
b'EPDPQ2UhDbb7HTpZKIWNH/kTD8ignHMIHO0lpCrH++GDkdm3Eq8UieF+CQq5c3KH99iOzyXnOjNFNrEWdcbS3eHXco6upMKY6EVrJGcZyuZONOJX1PIvTmsDFOYnHIta5+TZ4k4LiUb6010QBCHVvM1cGi6hQty35HdGSskbwjc='
Password do recetor: ········
Texto decifrado:
b'EPDPQ2UhDbb7HTpZKIWNH/kTD8ignHMIHO0lpCrH++GDkdm3Eq8UieF+CQq5c3KH99iOzyXnOjNFNrEWdcbS3eHXco6upMKY6EVrJGcZyuZONOJX1PIvTmsDFOYnHIta5+TZ4k4LiUb6010QBCHVvM1cGi6hQty35HdGSskbwjc='


Através de uma ligera comparação entre o texto antes de ser cifrado e depois de ser decifrado, pode-se verificar que é exatamente o mesmo.